## Exploratory Data Analysis (EDA) - Supermercados en Valencia:

Proyecto en el cual se comparan 3 de las principales cadenas de supermercados en Valencia (Mercadona, Consum y Carrefour) para dilucidar cual es la mejor para realizar las compras domésticas.

## Fase 1: Lectura y limpieza

In [1]:
# Importación de la librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go 
import warnings
warnings.filterwarnings("ignore")   # Restringir la aparicion de warnings

In [2]:
# Importar los datasets bajo el nombre df_supermercado_raw

# Datasets Mercadona
df_mercadona_raw = pd.read_csv("data/Raw_Data/Mercadona/mercadona_raw.csv")
df_mercadona_raw_2 = pd.read_csv("data/Raw_Data/Mercadona/products_macro_mercadona.csv")
df_mercadona_categorias = pd.read_csv("data/Raw_Data/Mercadona/mercadona_categorias_grupos.csv")

# Dataset Consum
df_consum_raw = pd.read_csv("data/Raw_Data/Consum/consum_raw.csv")

# Dataset Carrefour
df_carrefour_raw = pd.read_csv("data/Raw_Data/Carrefour/carrefour_raw.csv")

In [3]:
# Visualización del dataset para entenderlo y ver depuración de los datos necesita
df_mercadona_raw.head()

,elemento_id,texto_completo,nombre,precio,precio_unidad,marca,imagen_url,categoria_url,categoria_nombre
0,1,"Aceite de oliva 0,4º Hacendado\nGarrafa 5 L\n1...","Aceite de oliva 0,4º Hacendado","18,75 € /ud.",NaN,NaN,https://prod-mercadona.imgix.net/images/c17880...,https://tienda.mercadona.es/categories/112,112
1,3,"Aceite de oliva 0,4º Hacendado\nBotella 1 L\n3...","Aceite de oliva 0,4º Hacendado","3,85 € /ud.",NaN,NaN,https://prod-mercadona.imgix.net/images/65c234...,https://tienda.mercadona.es/categories/112,112
2,5,Aceite de oliva virgen extra Hacendado\nGarraf...,Aceite de oliva virgen extra Hacendado,"13,70 € /ud.",NaN,NaN,https://prod-mercadona.imgix.net/images/c6fc6d...,https://tienda.mercadona.es/categories/112,112
3,7,Aceite de oliva virgen extra Hacendado\nBotell...,Aceite de oliva virgen extra Hacendado,"4,65 € /ud.",NaN,NaN,https://prod-mercadona.imgix.net/images/bd2485...,https://tienda.mercadona.es/categories/112,112
4,9,Aceite de oliva virgen extra Hacendado Gran Se...,Aceite de oliva virgen extra Hacendado Gran Se...,"5,95 € /ud.",NaN,NaN,https://prod-mercadona.imgix.net/images/c60e6e...,https://tienda.mercadona.es/categories/112,112
